In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

In [2]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()
            
    
    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]), name='weights')
            b1 = tf.Variable(tf.zeros([num_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name='hidden1')
            
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name='weights')
            b0 = tf.Variable(tf.zeros([10]), name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
            
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("weights_hidden", w1)
        tf.summary.histogram("biases_hidden", b1)
        tf.summary.histogram("weights_output", w0)
        tf.summary.histogram("biases_output", b0)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
        
        
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.initialize_all_variables())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_sl_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

In [4]:
!rm -rf /tmp/mnist_sl_logs

In [5]:
nn = SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x: batch_xs, nn.t: batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run(
        [nn.summary, nn.loss, nn.accuracy],
        feed_dict={nn.x: mnist.test.images, nn.t: mnist.test.labels})
        print('Step: %d, Loss: %f, Accuracy: %f'
                     % (i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step: 100, Loss: 2840.777100, Accuracy: 0.916100
Step: 200, Loss: 2154.473633, Accuracy: 0.937200
Step: 300, Loss: 1840.163330, Accuracy: 0.943100
Step: 400, Loss: 1865.988770, Accuracy: 0.942700
Step: 500, Loss: 1440.675415, Accuracy: 0.958100
Step: 600, Loss: 1453.961792, Accuracy: 0.958500
Step: 700, Loss: 1300.038086, Accuracy: 0.962000
Step: 800, Loss: 1197.232422, Accuracy: 0.963200
Step: 900, Loss: 1218.235962, Accuracy: 0.964500
Step: 1000, Loss: 1193.530518, Accuracy: 0.964200
Step: 1100, Loss: 1151.622925, Accuracy: 0.964900
Step: 1200, Loss: 1128.236938, Accuracy: 0.965600
Step: 1300, Loss: 1074.782349, Accuracy: 0.967700
Step: 1400, Loss: 1091.455078, Accuracy: 0.966700
Step: 1500, Loss: 1018.512085, Accuracy: 0.970500
Step: 1600, Loss: 1039.968018, Accuracy: 0.970200
Step: 1700, Loss: 950.977417, Accuracy: 0.972700
Step: 1800, Lo